In [184]:
import pandas as pd
import numpy as np
from numpy import ma, atleast_2d, pi, sqrt, sum, transpose


def get_entropy(df, bins=None):
    pdf = pdf_histogram(df, bins)
    ## log base 2 returns H(X) in bits
    return -np.sum( pdf * ma.log2(pdf).filled(0)) 


def pdf_histogram(df, bins):

    ordered_bins = [sorted(bins[key]) for key in sorted(bins.keys())]
    hist, dedges = np.histogram(df.dropna().values, bins=2)#ordered_bins[0])
    return hist/hist.sum()


def sigma_bins(df, lags, max_bins=2):
    """ 
    Returns bins for N-dimensional data, using standard deviation binning: each 
    bin is one S.D in width, with bins centered on the mean. Where outliers exist 
    beyond the maximum number of SDs dictated by the max_bins parameter, the
    bins are extended to minimum/maximum values to ensure all data points are
    captured. This may mean larger bins in the tails, and up to two bins 
    greater than the max_bins parameter suggests in total (in the unlikely case of huge
    outliers on both sides). 
    Args:
        max_bins        -   (int)       The maximum allowed bins in each dimension
    Returns:
        bins            -   (dict)      The optimal bin-edges for pdf estimation
                                        using the histogram method, keyed by df column names
    """


    bins = {k:[np.mean(v)-int(max_bins/2)*np.std(v) + i * np.std(v) for i in range(max_bins)] for (k,v) in df.iteritems()}

    # Since some outliers can be missed, extend bins if any points are not yet captured
    [bins[k].append(df[k].min()) for k in df.keys() if df[k].min() < min(bins[k])]
    [bins[k].append(df[k].max()) for k in df.keys() if df[k].max() > max(bins[k])]

    bins.update({fieldname + '_lag' + str(t): edges          
                        for (fieldname, edges) in bins.items() for t in range(lags)})
    return bins


def non_linear_transfer_entropy(df: pd.DataFrame, endog: str, exog: str, lags=1):
    df = df[[exog, endog]]
    bins = sigma_bins(df, lag)

    for col_name in list(df.columns):
        for t in range(1, lag + 1):
            df[col_name + '_lag' + str(t)] = df[col_name].shift(t)

    ## Initialise list to return TEs
    transfer_entropies = [0, 0]

    ## Require us to compare information transfer bidirectionally
    for i, (X, Y) in enumerate({exog:endog, endog:exog}.items()):

        ### Entropy calculated using Probability Density Estimation:
            # Following: https://stat.ethz.ch/education/semesters/SS_2006/CompStat/sk-ch2.pdf
            # Also: https://www.cs.cmu.edu/~aarti/Class/10704_Spring15/lecs/lec5.pdf

        ## Note Lagged Terms
        X_lagged = X + '_lag' + str(lag - 1)
        Y_lagged = Y + '_lag' + str(lag - 1)

        ### Estimate PDF using Gaussian Kernels and use H(x) = p(x) log p(x)

        ## 1. H(Y,Y-t,X-t)  
        H1 = get_entropy(df = df[[Y, Y_lagged, X_lagged]], 
                        bins = {k:v for (k, v) in bins.items() if k in[Y, Y_lagged, X_lagged]})
        ## 2. H(Y-t,X-t)
        H2 = get_entropy(df = df[[X_lagged,Y_lagged]],
                        bins = {k:v for (k, v) in bins.items() if k in [X_lagged, Y_lagged]}) 
        ## 3. H(Y,Y-t)  
        H3 = get_entropy(df = df[[Y,Y_lagged]],
                        bins =  {k: v for (k, v) in bins.items() if k in [Y, Y_lagged]})
        ## 4. H(Y-t)  
        H4 = get_entropy(df = df[[Y_lagged]],
                        bins =  {k: v for (k, v) in bins.items() if k in [Y_lagged]})                


        ### Calculate Conditonal Entropy using: H(Y|X-t,Y-t) = H(Y,X-t,Y-t) - H(X-t,Y-t)
        conditional_entropy_joint =  H1 - H2

        ### And Conditional Entropy independent of X(t) H(Y|Y-t) = H(Y,Y-t) - H(Y-t)            
        conditional_entropy_independent = H3 - H4

        ### Directional Transfer Entropy is the difference between the conditional entropies
        transfer_entropies[i] =  conditional_entropy_independent - conditional_entropy_joint
        
    return transfer_entropies


def non_linear_transfer_entropy(df: pd.DataFrame, endog: str, exog: str, lags=1):
    df = df[[exog, endog]]
    bins = sigma_bins(df, lag)

    for col_name in list(df.columns):
        for t in range(1, lag + 1):
            df[col_name + '_lag' + str(t)] = df[col_name].shift(t)

    ## Initialise list to return TEs
    transfer_entropies = [0, 0]

    ## Require us to compare information transfer bidirectionally
    for i, (X, Y) in enumerate({exog:endog, endog:exog}.items()):

        ### Entropy calculated using Probability Density Estimation:
            # Following: https://stat.ethz.ch/education/semesters/SS_2006/CompStat/sk-ch2.pdf
            # Also: https://www.cs.cmu.edu/~aarti/Class/10704_Spring15/lecs/lec5.pdf

        ## Note Lagged Terms
        X_lagged = X + '_lag' + str(lag - 1)
        Y_lagged = Y + '_lag' + str(lag - 1)

        ### Estimate PDF using Gaussian Kernels and use H(x) = p(x) log p(x)

        ## 1. H(Y,Y-t,X-t)  
        H1 = get_entropy(df = df[[Y, Y_lagged, X_lagged]], 
                        bins = {k:v for (k, v) in bins.items() if k in[Y, Y_lagged, X_lagged]})
        
        ## 2. H(Y-t,X-t)
        lagged_bins = bins = {k:v for (k, v) in bins.items() if k in [X_lagged, Y_lagged]}
        H2 = get_entropy(df = df[[X_lagged,Y_lagged]], bins = lagged_bins)

        ## 3. H(Y,Y-t)
        lagged_bins = {k: v for (k, v) in bins.items() if k in [Y, Y_lagged]}
        H3 = get_entropy(df = df[[Y,Y_lagged]], bins = lagged_bins)
        ## 4. H(Y-t)  
        lagged_bins =  {k: v for (k, v) in bins.items() if k in [Y_lagged]}
        H4 = get_entropy(df = df[[Y_lagged]], bins = lagged_bins)                


        ### Calculate Conditonal Entropy using: H(Y|X-t,Y-t) = H(Y,X-t,Y-t) - H(X-t,Y-t)
        conditional_entropy_joint =  H1 - H2

        ### And Conditional Entropy independent of X(t) H(Y|Y-t) = H(Y,Y-t) - H(Y-t)            
        conditional_entropy_independent = H3 - H4

        ### Directional Transfer Entropy is the difference between the conditional entropies
        transfer_entropies[i] =  conditional_entropy_independent - conditional_entropy_joint
    return transfer_entropies

df = pd.read_csv("data/results_day_binned_with_states.csv")
endog = 'Anti-Regulation Fear-of-Regulation'
exog = 'Daily Background Checks'
df = df[[exog, endog]]

print(non_linear_transfer_entropy(df, endog, exog, lags = 1))

[0.09885128772440951, -0.014221854912478515]


In [185]:
variables = [
    'Pro-Regulation',
    'Anti-Regulation',
    'Self-Defense',
    'Fear-of-Regulation',
    'Pro-Regulation Self-Defense',
    'Anti-Regulation Self-defense',
    'Pro-Regulation Fear-of-Regulation',
    'Anti-Regulation Fear-of-Regulation',
    'Daily Background Checks'
]

df = pd.read_csv("data/results_day_binned_with_states.csv")

for endog in variables:
    for exog in variables:
        if endog == exog:
            continue
        te = non_linear_transfer_entropy(df, endog, exog)
        print(te)

<ipython-input-184-d288a73b6c49>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name + '_lag' + str(t)] = df[col_name].shift(t)


[-0.02091194434190466, -0.02091194434190466]
[-0.01102327025004965, -0.01102327025004965]
[-0.01102327025004965, -0.01102327025004965]
[-0.02091194434190466, -0.02091194434190466]
[-0.032248324162314296, -0.00657082565865745]
[-0.032248324162314296, -0.00657082565865745]
[-0.01102327025004965, -0.01102327025004965]
[0.09885128772440951, -0.014221854912478515]
[-0.02091194434190466, -0.02091194434190466]
[-0.02091194434190466, -0.02091194434190466]
[-0.02091194434190466, -0.02091194434190466]
[-0.032248324162314296, -0.00657082565865745]
[-0.032248324162314296, -0.00657082565865745]
[-0.032248324162314296, -0.00657082565865745]
[-0.02091194434190466, -0.02091194434190466]
[0.09885128772440951, -0.014221854912478515]
[-0.01102327025004965, -0.01102327025004965]
[-0.02091194434190466, -0.02091194434190466]
[-0.01102327025004965, -0.01102327025004965]
[-0.02091194434190466, -0.02091194434190466]
[-0.032248324162314296, -0.00657082565865745]
[-0.032248324162314296, -0.00657082565865745]
[-0

In [181]:
variables = [
    'Pro-Regulation',
    'Anti-Regulation',
    'Self-Defense',
    'Fear-of-Regulation',
    'Pro-Regulation Self-Defense',
    'Anti-Regulation Self-defense',
    'Pro-Regulation Fear-of-Regulation',
    'Anti-Regulation Fear-of-Regulation',
    'Daily Background Checks'
]

df = pd.read_csv("data/results_day_binned_with_states.csv")
#endog = 'Anti-Regulation Fear-of-Regulation'
#exog = 'Daily Background Checks'
#df = df[[exog, endog]]
df = df[variables]

lags = 1
max_bins = 2
bins = {k:[np.mean(v)-int(max_bins/2)*np.std(v) + i * np.std(v) for i in range(max_bins)] for (k,v) in df.iteritems()}   # Note: same as:  self.df.to_dict('list').items()}

# Since some outliers can be missed, extend bins if any points are not yet captured
[bins[k].append(df[k].min()) for k in df.keys() if df[k].min() < min(bins[k])]
[bins[k].append(df[k].max()) for k in df.keys() if df[k].max() > max(bins[k])]

# bins.update({fieldname + '_lag' + str(t): edges          
#                     for (fieldname, edges) in bins.items() for t in range(lags)})
ordered_bins = [sorted(bins[key]) for key in sorted(bins.keys())]
print(bins.keys())
print(ordered_bins)
hist, dedges = np.histogram(df.values, 2)
print(hist/hist.sum(), hist)

dict_keys(['Pro-Regulation', 'Anti-Regulation', 'Self-Defense', 'Fear-of-Regulation', 'Pro-Regulation Self-Defense', 'Anti-Regulation Self-defense', 'Pro-Regulation Fear-of-Regulation', 'Anti-Regulation Fear-of-Regulation', 'Daily Background Checks'])
[[-408.29275405722103, 153.3295454545455, 4119], [-233.65133299132862, 90.94318181818181, 2408], [-111.2910499512158, 39.89772727272728, 1078], [29745, 57396.51577124973, 74427.71590909091, 116161], [-274.74096280376943, 104.34090909090907, 2824], [-226.61685153916738, 87.26136363636363, 2372], [-43.484059734240205, 16.909090909090907, 441], [-134.18886826562644, 51.25, 1391], [-242.96727587845243, 90.30681818181819, 2455]]
[0.91035354 0.08964646] [721  71]


In [95]:
def shuffle_series(DF, only=None):
    """
    Function to return time series shuffled rowwise along each desired column. 
    Each column is shuffled independently, removing the temporal relationship.
    This is to calculate Z-score and Z*-score. See P. Boba et al (2015)
    Calculated using:       df.apply(np.random.permutation)
    Arguments:
        df              -   (DataFrame) Time series data 
        only            -   (list)      Fieldnames to shuffle. If none, all columns shuffled 
    Returns:
        df_shuffled     -   (DataFrame) Time series shuffled along desired columns    
    """
    if not only == None:
        shuffled_DF = DF.copy()
        for col in only:
            series = DF.loc[:, col].to_frame()
            shuffled_DF[col] = series.apply(np.random.permutation)
    else:
        shuffled_DF = DF.apply(np.random.permutation)
    
    return shuffled_DF


def significance(df, TE, endog, exog, lag=5, n_shuffles=100):
        """
        Perform significance analysis on the hypothesis test of statistical causality, for both X(t)->Y(t)
        and Y(t)->X(t) directions
   
        Calculated using:  Assuming stationarity, we shuffle the time series to provide the null hypothesis. 
                           The proportion of tests where TE > TE_shuffled gives the p-value significance level.
                           The amount by which the calculated TE is greater than the average shuffled TE, divided
                           by the standard deviation of the results, is the z-score significance level.
        Arguments:
            TE              -      (list)    Contains the transfer entropy in each direction, i.e. [TE_XY, TE_YX]
            endog           -      (string)  The endogenous variable in the TE analysis being significance tested (i.e. X or Y) 
            exog            -      (string)  The exogenous variable in the TE analysis being significance tested (i.e. X or Y) variables (giving z*-score)  
        Returns:
            p_value         -      Probablity of observing the result given the null hypothesis
            z_score         -      Number of Standard Deviations result is from mean (normalised)
        """ 

        ## Prepare array for Transfer Entropy of each Shuffle
        shuffled_TEs = np.zeros(shape = (2, n_shuffles))

        for i in range(n_shuffles):
                ## Perform Shuffle
                df = shuffle_series(df)
                
                ## Calculate New TE
                
                transfer_enthropy_shuffled = non_linear_transfer_entropy(df, endog, exog)
                shuffled_TEs[:,i] = transfer_enthropy_shuffled

        
        ## Calculate p-values for each direction
        p_values = (np.count_nonzero(TE[0] < shuffled_TEs[0,:]) /n_shuffles , \
                    np.count_nonzero(TE[1] < shuffled_TEs[1,:]) /n_shuffles)

        ## Calculate z-scores for each direction
        z_scores = ( ( TE[0] - np.mean(shuffled_TEs[0,:]) ) / np.std(shuffled_TEs[0,:]) , \
                     ( TE[1] - np.mean(shuffled_TEs[1,:]) ) / np.std(shuffled_TEs[1,:])  )
        
        TE_mean = ( np.mean(shuffled_TEs[0,:]), \
                     np.mean(shuffled_TEs[1,:]) )
        
        ## Return the self.DF value to the unshuffled case
        return p_values, z_scores, TE_mean


In [153]:
endog = 'Anti-Regulation Fear-of-Regulation'
exog = 'Daily Background Checks'
te = non_linear_transfer_entropy(df, endog, exog)
print(te, significance(df, te, endog, exog))

[0.0, 0.0] ((0.0, 0.0), (nan, nan), (0.0, 0.0))


<ipython-input-95-2504a8a586a0>:60: RuntimeWarning: invalid value encountered in double_scalars
  z_scores = ( ( TE[0] - np.mean(shuffled_TEs[0,:]) ) / np.std(shuffled_TEs[0,:]) , \
<ipython-input-95-2504a8a586a0>:61: RuntimeWarning: invalid value encountered in double_scalars
  ( TE[1] - np.mean(shuffled_TEs[1,:]) ) / np.std(shuffled_TEs[1,:])  )


In [152]:
endog = 'Anti-Regulation Fear-of-Regulation'
exog = 'Daily Background Checks'
te = non_linear_transfer_entropy(df, endog, exog)
print(te, significance(df, te, endog, exog))

[0.0, 0.0] ((0.0, 0.0), (nan, nan), (0.0, 0.0))


<ipython-input-95-2504a8a586a0>:60: RuntimeWarning: invalid value encountered in double_scalars
  z_scores = ( ( TE[0] - np.mean(shuffled_TEs[0,:]) ) / np.std(shuffled_TEs[0,:]) , \
<ipython-input-95-2504a8a586a0>:61: RuntimeWarning: invalid value encountered in double_scalars
  ( TE[1] - np.mean(shuffled_TEs[1,:]) ) / np.std(shuffled_TEs[1,:])  )


In [154]:
variables = [
    'Pro-Regulation',
    'Anti-Regulation',
    'Self-Defense',
    'Fear-of-Regulation',
    'Pro-Regulation Self-Defense',
    'Anti-Regulation Self-defense',
    'Pro-Regulation Fear-of-Regulation',
    'Anti-Regulation Fear-of-Regulation',
    'Daily Background Checks'
]

for endog in variables:
    for exog in variables:
        if endog == exog:
            continue
        te = non_linear_transfer_entropy(df, endog, exog)
        p, z, mean =  significance(df, te, endog, exog)
        if p[0] < 0.1 or p[1] < 0.1:
            print(f'X = {endog} Y = {exog}, p-values =  {p}, transfer entropy =  {te}')

KeyError: "None of [Index(['Anti-Regulation', 'Pro-Regulation'], dtype='object')] are in the [columns]"